<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_NLP/blob/main/cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install xlrd
!pip install openpyxl

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 450kB 35.5MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
#파일 업로드
from google.colab import files

#파일 읽기
#엑셀 및 txt 파일 읽기 후에 클래스 작성
# uploaded = files.upload()

Saving sample_data_processed_4600.xlsx to sample_data_processed_4600.xlsx


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from tqdm.notebook import tqdm
from openpyxl import Workbook
import numpy as np
import pandas as pd
import copy


In [ ]:
#TF IDF 를 이용한  Cosine 유사도 판별

class Similarity:

  ######################
  #  코사인 유사도 판별
  #  text : 질문
  #  questionList : DaraFrame의 질문목록
  #  excelDF : 읽어온 엑셀 DataFrame
  #######################

  def __init__(self, text, excelDF, index):
    self.text = text # 사용자 질문
    self.excelDF = excelDF
    self.setQuestion(index) #question 벡터화를 위한 question 리스트

  #question 초기화 함수, 카테고리 나누기
  def setQuestion(self, index):
    question = []
    df_np = pd.DataFrame.to_numpy(self.excelDF)
    for i in range(1, len(df_np)):
      question.append(df_np[i][index])
    self.questionList = question
    
    
  def vecotrization(self): #벡터화

    ## 후에 DB에서 해당 벡터요소를 가져오는 작업 추가 

    vectorizer = TfidfVectorizer() 
    TFIDF_matrix = copy.deepcopy(self.questionList) # 리스트 깊은 복사 수행
    TFIDF_matrix.append(self.text) # 리스트에 추가
    text_matrix = vectorizer.fit_transform(TFIDF_matrix) #벡터화 실시

    return text_matrix

  def get_recommendation(self, row):
    new_matrix = self.vecotrization() #새로운 메트릭스 생성
    cosine_sim = linear_kernel(new_matrix, new_matrix) # 코사인 유사도 계산

    # 수정 필요 ##

    #추가된 마지막 요소의 cosine_sim 의 list 가져온다
    sim_scores = list(enumerate(cosine_sim[-1]))

    # 유사도에 따라질문들을 정렬.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 질문의 인덱스
    test_ind = [i[0] for i in sim_scores]

    # print(self.excelDF['질문'].iloc[test_ind])
    # print(self.excelDF[row].iloc[test_ind])

    # 가장 유사한 10개의 열 을 리턴
    return self.excelDF[row].iloc[test_ind]


  

In [ ]:
def main():

  # path = '/content/sample_data_processed_4600.xlsx'
  path = '/content/newxlsx.xlsx'
  rowName = '질문'
  df = pd.read_excel(path)

  text = "피임약을 먹고 부정출혈이 났어요 ㅠㅠ 어찌해야하나요"

  # text, dataframe, 코사인 유사도 계산할 df 의 열 index
  # original data index = 0
  s = Similarity(text, df, 0) 
  a = s.get_recommendation(rowName)
  print(a) #답변 출력

if __name__ == '__main__':
  main()


788      피임약 5팩째 복용 중입니다 스마일라식을 하면 아래 있는 안약을 넣어야 하는데 피...
376      생리예정일 18일 정도 지난 후 피가 나와서 피임 목적으로 경구피임약을 처음으로 ...
589      안녕하세오. 저는 생리기간이 일정한 편 애속하엿습니다. 10월 31일에 돤계를 맺...
738      제가 8월 3일에 생리 시작하고 10일에 끝났는데요 그리고 22일 갈색 분미물이 ...
1902     이번 달에 생리를 두 번 했는데 (1월 4일-1월 9일 & 1월 17일-20일) ...
3058     4월부터 꾸준히 피임약을 복용해오던 사람입니다. 그런데 8월 초에 휴약기에 접어들...
130      피임약은 기존에 계속 먹고 있었고.. 감기약이랑 시간차를 두고 먹었어야 했는데 까...
1128     피임약 휴약기에 생리가 없어서 피임이 필요한 경우가 아니라면 중단하라는 답변을 받...
913      생리 첫날부터 경구 피임약 멜리안정을 복용했고 14일째 되는 날 낮 2시경에 14...
1505     제목 그래도 입니다. 현재 경구피임약 다온을 세 달째 복용 중입니다. 첫 달에는 ...
Name: 질문, dtype: object
788      피임약 5팩째 복용 중입니다 스마일라식을 하면 아래 있는 안약을 넣어야 하는데 피...
376      생리예정일 18일 정도 지난 후 피가 나와서 피임 목적으로 경구피임약을 처음으로 ...
589      안녕하세오. 저는 생리기간이 일정한 편 애속하엿습니다. 10월 31일에 돤계를 맺...
738      제가 8월 3일에 생리 시작하고 10일에 끝났는데요 그리고 22일 갈색 분미물이 ...
1902     이번 달에 생리를 두 번 했는데 (1월 4일-1월 9일 & 1월 17일-20일) ...
3058     4월부터 꾸준히 피임약을 복용해오던 사람입니다. 그런데 8월 초에 휴약기에 접어들...
130      피임약은 기존에 계속 먹고 있었고.. 감기약이랑 시간차를 두고 먹었어야

***피임약과 관련없는 데이터 삭제*** (1) : np array에서 삭제


In [ ]:
#filtering

#엑셀 파일 중 피임약 포함 안되는 Q&A 삭제, 새로운 엑셀 만들어서 다시 확인
path = '/content/sample_data_processed_4600.xlsx'

#Excel 읽고 numpy array로 변경
excel = pd.read_excel(path)
excelNP = pd.DataFrame.to_numpy(excel)

#피임약과 관련된 단어
birthPillWord = ['피임약', '머시론', '야즈', '야스민', '피임 약', '다온', '센스리베', '센스 리베', '멜리안', '마이보라', '경구피임', '엘라원', '머 시론'
                , '피암', '노레보', '엘리원', '피임목적', '엘라', '경구 피임제', '머*론', '트*페린', '엘*원', '미뉴렛', '미 뉴렛', '머시', '노레*'
                , '센스데이', '센스 데이'
                ]

WrongQandA = [] # 피임약이 존재하지 않는 리스트
newExcelNP = [] # 피임약이 존재하는 리스트


for i in tqdm(range(len(excelNP))):
  
  if excelNP[i] is None : break #test

  #질문에 피임약에 관련된 단어들 제외
  if any(ele in excelNP[i][0] for ele in birthPillWord):
    newExcelNP.append(excelNP[i])
    continue
  #답변에 피임약 관련된 단어들 존재할 시 제외 
  elif any(ele in excelNP[i][1] for ele in birthPillWord):
    newExcelNP.append(excelNP[i])
    continue
  else:
    WrongQandA.append(excelNP[i])
    

 

***피임약과 관련없는 데이터 삭제*** (2) : 엑셀에 저장

In [ ]:
#index 82, index 176 (답변 작성 x), 

#br 도 빼기



def makeXlsxFile(path, name, list):
  wb = Workbook()
  filename = path + name

  sheet = wb.active
  sheet.append(['질문', '답변'])
  for x in tqdm(list):
    sheet.append([x[0], x[1]])
    wb.save(filename)

path = '/content/'
name = 'newxlsx.xlsx'

makeXlsxFile(path, name, newExcelNP)